<a href="https://colab.research.google.com/github/adupatil/Hindi-to-English-Translator/blob/main/NLP_Translation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install pyenchant

In [ ]:
import pandas as pd
import numpy as np
#import enchant as enc
import re
from sklearn.model_selection import train_test_split
import torch

In [ ]:
d = enc.Dict("en_US")

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLP_Project/hindi_english_parallel.csv")

In [ ]:
data.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [ ]:
data.dropna(inplace=True)

In [ ]:
data['Length'] = data['hindi'].apply(lambda x : len(x.split()))

In [ ]:
data[data['Length'] == data['Length'].max()]

,hindi,english,Length
1491388,"134, 135 विजय 6 क्षेत्रफल, ओड़िसा का 2 कांची-व...","Puranic 70 'Goti' system, a variety of serfdom...",1380


In [ ]:
def check_eng(col):
  L = col.split()
  for i in L:
    if (d.check(i)==True) and (i.isnumeric()==False):
      return True
  return False

In [ ]:
data['Eng_in_Hind'] = data['hindi'].apply(check_eng)

In [ ]:
data.Eng_in_Hind.sum()

68839

In [ ]:
data[data['Eng_in_Hind'] == True].tail(20)

,hindi,english,Length,Eng_in_Hind
1561034,अंतरराष्ट्रीय मंच पर प्रभावी भागीदार के तौर पर...,With India emerging as an influential player i...,37,True
1561042,"अनुच्छेद 14,15 और 16 द्वारा परिवर्द्धित, स्थित...","Within the same ambit, but distinct from it, i...",24,True
1561055,जहां वर्ष 2016 में आर एण्ड डी में निवेश का बाज...,With our R&amp;D market valued at US$ 22.3 bil...,17,True
1561117,2013-14 में विकास दर के 4.9 प्रतिशत के अनुमान ...,With the growth rate in 2013-14 estimated at 4...,40,True
1561253,वर्ष 2014 में महिला सांसदों की संख्या कुल सदस्...,Women MPs constituted 12.15% of the total in 2...,14,True
1561273,"राष्ट्रपति ने कहा, हमारे करोड़ों देशवासियों को...",WORK TOWARDS A TECHNOLOGICAL REVOLUTION AIMED ...,22,True
1561274,महिलाओं के बारे में नकारात्मक धारणाओं को बदलने...,WORK TOWARDS CHANGING NEGATIVE PERCEPTIONS ABO...,20,True
1561275,"राष्ट्रपति मुखर्जी ने ताजिक राष्ट्रपति से कहा,...",WORLD AND REGION FACES GROWING THREAT OF TERRO...,23,True
1561287,‘आवासी’ लेखकों एवं कलाकारों ने राष्ट्रपति जी स...,WRITERS AND ARTISTS ‘IN-RESIDENCE’ CALL ON THE...,14,True
1561366,"(x) [वित्त (सं. 2) अधिनियम, 1998 द्वारा 1.4.19...","(x) [Omitted by the Finance (No. 2) Act, 1998...",12,True


In [ ]:
def pre_processing(sentence):
  #remove % s 
  sentence = re.sub(r"\ss\s", "", sentence)
  sentence = re.sub(r"\sd\s", "", sentence)

  # remove rolename
  sentence = re.sub(r"\brolename\b","",sentence)
  sentence = re.sub(r"\num\b","",sentence)

  #remove (_ C) and (M _)
  pattern = r"\s*\(\_\s*[a-zA-Z]\)"
  sentence = re.sub(pattern, "", sentence)
  pattern = r"\s*\([a-zA-Z]\s_\)"
  sentence = re.sub(pattern, "", sentence)

  sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)

  #remove % [a-z]
  #sentence = re.sub(r'%\s*[a-z]', '', sentence)

  #replace c_lean to clean and Refresh _ Node  to Refresh Node
  if re.search(r'[a-z]_[a-z]', sentence):
      sentence = re.sub(r'_', "", sentence)
  if re.search(r'[a-z]_[A-Z]', sentence):
      sentence = re.sub(r'_', " ", sentence)
  if re.search(r'[a-zA-Z]\s_\s[a-z]', sentence):
      sentence = re.sub(r'\s_\s', "", sentence)
  if re.search(r'[a-z]\s_\s[A-Z]', sentence):
      sentence = re.sub(r'\s_\s', " ", sentence)

  # replacing everything with space except (".", "?", "!", ",","¿",":","%","|","(",")","-","<",">","।(hindi full stop)","॰")
  sentence = re.sub(r"[-?.!,¿/:|%()<>।॰{}#_'\"]", " ", sentence)
  sentence = re.sub(r"…", " ", sentence)
  
  #remove double quotes
  sentence = re.sub(r'"', " ", sentence)
  sentence = re.sub(r"…", " ", sentence)
  
  #remove english letters
  #sentence = re.sub(r'[a-zA-z]', " ", sentence)
  
  #remove numbers
  #sentence = re.sub(r'[0-9]', "", sentence)

  sentence = re.sub(r'#([^s]+)', r'1', sentence)
  
  #remove website links
  sentence = re.sub('((www.[^s]+)|(https?://[^s]+))','',sentence)
  
  #remove @anythin here
  sentence = re.sub('@[^s]+','',sentence)
  
  #remove multiple spaces
  sentence = re.sub(r'[" "]+', " ", sentence)
  
  # remove extra space
  sentence = sentence.strip()

  return sentence

In [ ]:
from time import time
t1 = time()
data['hindi'] = data['hindi'].apply(lambda x: pre_processing(x))
data['english'] = data['english'].apply(lambda x: pre_processing(x))
print("Time taken:",time()-t1)

Time taken: 189.76392936706543


In [ ]:
data[data['Eng_in_Hind'] == True].head(20)

,hindi,english,Length,Eng_in_Hind
121,जिस पर कार्रवाई हो सकती है वह न तो फोकस करने य...,actionable is not focusable or selectable,18,True
122,अंतर्क्रियात्मक कार्रवाई करने योग्य नहीं है,interactive is not actionable,7,True
124,का कोई पाठ अंतराफलक नहीं है,has no text interface,8,True
125,अभिभावक में सूची शिशु सूची सेमेल नहीं खाती,index in parent does not match child index,9,True
126,संबंध के लिए अन्योन्य गायब है,Missing reciprocal for relation,8,True
127,गायब नाम या सूचक,missing name or label,6,True
128,फोकस योग्य में सारणी अंतराफलक है कोई चयनीय अंत...,focusable has table interface no selection int...,18,True
129,में स्थिति है पर उसके साथ स्थिति नहीं है,has state without state,13,True
130,किसी समुच्चय का सदस्य नहीं है,does not belong to a set,8,True
131,rolenameसूची numपंक्ति और स्तंभ से मेल नहीं खा...,rolenameindex numdoes not match row and column,17,True


In [ ]:
drop_idx = data[data['hindi'].str.contains('rolename')].index.tolist()

In [ ]:
  # x = 'rolenameसूची numपंक्ति और स्तंभ से मेल नहीं खातेparent indexrow and column index'
  # x = re.sub(r"rolename","",x)
  # x = re.sub(r"num","",x)
  # x = re.sub(r"[a-zA-Z]","",x)

In [ ]:
data.drop(drop_idx,inplace=True)

---
## Dummy Code
---

In [ ]:
! pip install transformers
!pip install sentencepiece
#! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

In [ ]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.1, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 21017
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 189156
    })
})

In [ ]:
max_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [ ]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

In [ ]:
type(tokenized_datasets['train'])

dict

---
## Preparing data to feed the model

---



In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-hi-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [ ]:
input = data['hindi'][0]
output = data['english'][0]

inputs = tokenizer(input,text_target=output)

In [ ]:
inputs

{'input_ids': [62, 4414, 21, 8765, 13268, 488, 22086, 30, 2036, 824, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [3564, 78, 4315, 138, 47442, 333, 4373, 0]}

In [ ]:
max_length = 256

def create_tokens(col):
    inputs = col['hindi']
    targets = col['english']
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [ ]:
import pyarrow as pa

In [ ]:
## Converting df to arrow
table = pa.Table.from_pandas(data.iloc[:10])

In [ ]:
train = data.sample(frac=0.9)
validation = data.drop(train.index)

In [ ]:
train.head()

,hindi,english
1402830,अपने लोगों के पापों के लिए गांधीजी अनशन कर रहे...,Gandhiji was on fast for the sins of bis peopl...
1538052,लोकहित के मामले उठाने हेतु प्रक्रियात्मक उपाय,Procedural Devices for raising matters of publ...
1300648,इसके अलावा हम यह भी अक्सर देखते हैं कि वर्षा क...,Further we often observe that the water level ...
612557,सी डी रॉम का कोष पढ़ने में असफल,Unable to read the cdrom database s
259944,उन्होंने कहा ऐ मेरी क़ौम के लोगो हमने एक किताब...,They said “O our people We have indeed heard a...


In [ ]:
## Demo output
train.iloc[:10].apply(create_tokens,axis=1)[1402830]

{'input_ids': [62, 87, 6, 3865, 6, 46, 59164, 4958, 4577, 9261, 64, 153, 113, 7, 117, 11734, 1751, 4754, 93, 6, 46, 172, 743, 165, 64, 153, 113, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [38160, 11614, 72, 70, 2836, 34, 4, 1350, 8, 11312, 23, 120, 10, 59, 1848, 3405, 796, 10, 222, 11, 4934, 727, 0]}

In [ ]:
t1 = time()
train_tokens = train.iloc[:30000].apply(create_tokens,axis=1)
validation_tokens = validation.iloc[:5000].apply(create_tokens,axis=1)
print(f"Time taken to tokenize:{time()-t1}")

Time taken to tokenize:11.991955995559692


In [ ]:
# Saved on Drive
train_tokens.to_csv("/content/drive/MyDrive/NLP_Project/datasets/train_tokens_subset.csv",index=False)
validation_tokens.to_csv("/content/drive/MyDrive/NLP_Project/datasets/validation_tokens_subset.csv",index=False)

In [ ]:
train_tokens.iloc[0]

{'input_ids': [62, 87, 6, 3865, 6, 46, 59164, 4958, 4577, 9261, 64, 153, 113, 7, 117, 11734, 1751, 4754, 93, 6, 46, 172, 743, 165, 64, 153, 113, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [38160, 11614, 72, 70, 2836, 34, 4, 1350, 8, 11312, 23, 120, 10, 59, 1848, 3405, 796, 10, 222, 11, 4934, 727, 0]}

In [ ]:
# Loading saved tokenized data
train_tokens = pd.read_csv("/content/drive/MyDrive/NLP_Project/datasets/train_tokens_subset.csv")
validation_tokens = pd.read_csv("/content/drive/MyDrive/NLP_Project/datasets/validation_tokens_subset.csv")

In [ ]:
train_tokens.rename(columns={'0':'train'},inplace=True)
validation_tokens.rename(columns={'0':'validate'},inplace=True)

In [ ]:
## Using eval to convert the dictionaries saved as str in dataframe to type dict
train_tokens['train'] = train_tokens['train'].apply(lambda x : eval(x))
validation_tokens['validate'] = validation_tokens['validate'].apply(lambda x : eval(x))

In [ ]:
type(train_tokens['train'].iloc[0])

dict

In [ ]:
## Converting to dictionary for feeding it to data collator
train_dict = train_tokens.to_dict()
validation_dict = validation_tokens.to_dict()

---
## Data Collation
----

In [ ]:
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator([train_dict['train'][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
## Padded values are alloted -100 so that it is not considered in the loss function
batch['labels']

tensor([[ 9757,   867,   288, 26855, 30741,    34,  8338,  1844,     8,  2156,
          2236,     0,  -100,  -100,  -100,  -100,  -100],
        [ 7557,    75,  1009,  3174,    26,     4,   752,  3992,    19,     4,
           415,  3064,   202,   312,     4, 18456,     0]])

In [ ]:
batch['decoder_input_ids']

tensor([[61126,  9757,   867,   288, 26855, 30741,    34,  8338,  1844,     8,
          2156,  2236,     0, 61126, 61126, 61126, 61126],
        [61126,  7557,    75,  1009,  3174,    26,     4,   752,  3992,    19,
             4,   415,  3064,   202,   312,     4, 18456]])

---
## Metrics 
---

In [ ]:
!pip install sacrebleu
!pip install evaluate

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
## Sample output for the BLEU metric
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

----
## Fine-Tuning Model
----

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
## Define the training arguments
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"finetuned-hi-to-en",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [ ]:
## Creating the trainer
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dict['train'],
    eval_dataset=validation_dict["validate"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Clean file pytorch_model.bin:   0%|          | 1.00k/288M [00:00<?, ?B/s]

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
## Evaluating the pre-trained model
trainer.evaluate(max_length=256)

{'eval_loss': 1.6579340696334839,
 'eval_bleu': 38.10873827589208,
 'eval_runtime': 152.4842,
 'eval_samples_per_second': 6.558,
 'eval_steps_per_second': 0.413}

In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
## Training 
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.648700
1000,3.237200
1500,3.104300


Adding files tracked by Git LFS: ['source.spm', 'target.spm']. This may take a bit of time if the files are large.


TrainOutput(global_step=1875, training_loss=3.2479682291666667, metrics={'train_runtime': 468.8841, 'train_samples_per_second': 63.982, 'train_steps_per_second': 3.999, 'total_flos': 512178870288384.0, 'train_loss': 3.2479682291666667, 'epoch': 3.0})

In [ ]:
trainer.evaluate(max_length=256)

{'eval_loss': 1.4444315433502197,
 'eval_bleu': 29.4654563867538,
 'eval_runtime': 248.9063,
 'eval_samples_per_second': 4.018,
 'eval_steps_per_second': 0.253,
 'epoch': 3.0}

In [ ]:
trainer.push_to_hub(tags="translation-hi-en", commit_message="Training complete")

To https://huggingface.co/Adwait1199/finetuned-hi-to-en
   9ef6e28..4e77c89  main -> main

   9ef6e28..4e77c89  main -> main

To https://huggingface.co/Adwait1199/finetuned-hi-to-en
   4e77c89..4f49761  main -> main

   4e77c89..4f49761  main -> main



'https://huggingface.co/Adwait1199/finetuned-hi-to-en/commit/4e77c89047a7ec3f5d0eb7a0b19cc31c4b22ab35'

---
## Pipeline and DataLoader
---

In [ ]:
## Converting token_dict to torch tensor
# def convert_to_tensor(in_dict):
#   train_examples = []
#   for item in in_dict['train']:
#       in_tensor = torch.Tensor(item['input_ids']).int()
#       out_tensor = torch.Tensor(item['output'])
#       train_examples.append([in_tensor, out_tensor])

#   out_dict = {'train': []}
#   for item in train_examples:
#       out_dict['train'].append({
#           'input': item[0].tolist(),
#           'output': item[1].tolist()
#       })

#   print(out_dict)

In [ ]:
from torch.utils.data import DataLoader
import torch 
train_dataloader = DataLoader(
    train_dict["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_dict["validate"], collate_fn=data_collator, batch_size=8
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

In [ ]:
! pip install accelerate

In [ ]:
from accelerate import Accelerator
from transformers import AdamW

## creating optimizer object
optimizer = AdamW(model.parameters(), lr=2e-5)

## defining accelerator
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "hi-en-translate-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

'Adwait1199/hi-en-translate-accelerate'

In [ ]:
output_dir = "/content/drive/MyDrive/NLP_Project/model/hindi-english-translator"

The first thing to note is that we use the generate() method to compute predictions, but this is a method on our base model, not the wrapped model Accelerate created in the prepare() method. That’s why we unwrap the model first, then call this method.

The second thing is that, like with token classification, two processes may have padded the inputs and labels to different shapes, so we use accelerator.pad_across_processes() to make the predictions and labels the same shape before calling the gather() method. If we don’t do this, the evaluation will either error out or hang forever.

In [ ]:
## postprocess() function takes predictions and labels and converts them to the lists of strings our metric object will expect

def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)

  0%|          | 0/11250 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

epoch 0, BLEU score: 73.13


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 1, BLEU score: 73.08


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 2, BLEU score: 74.50


---
## Saving model
---

In [ ]:
! pip install sacremoses

In [ ]:
torch.save(unwrapped_model,"/content/drive/MyDrive/NLP_Project/model/unwrapped_model.pt")

In [ ]:
loaded_model = torch.load("/content/drive/MyDrive/NLP_Project/model/unwrapped_model.pt").to(device)

In [ ]:
from transformers import pipeline

translator = pipeline(task="translation",tokenizer="Helsinki-NLP/opus-mt-hi-en",model=loaded_model,device=0)
translator(data.iloc[10]['hindi'])

[{'translation_text': 'Color and transparency of the highlighted whole being'}]

In [ ]:
data.iloc[10]['hindi']

'हाइलाइट किया गया भराई का रंग और पारदर्शिता। '

In [ ]:
article1 = ''' भारत ने प्रशासित खुराक की कुल संख्या में संयुक्त राज्य अमेरिका को पीछे छोड़ते हुए, COVID-19 टीकों की 100 मिलियन से अधिक खुराक दी है।
भारत सरकार ने फेसबुक और ट्विटर जैसे सोशल मीडिया प्लेटफॉर्म की निगरानी और नियमन के लिए नए नियमों का प्रस्ताव किया है।
अंतर्राष्ट्रीय मुद्रा कोष की एक हालिया रिपोर्ट के अनुसार, 2020 में महामारी-प्रेरित मंदी के बाद 2021 में भारत की अर्थव्यवस्था में मजबूती से उछाल आने की उम्मीद है।
भारत में किसान नए कृषि कानूनों का विरोध करना जारी रखते हैं, उनका दावा है कि इससे उनकी आजीविका को नुकसान पहुंचेगा और बड़े निगमों को लाभ होगा।
भारतीय अंतरिक्ष अनुसंधान संगठन ने हाल ही में एक उपग्रह लॉन्च किया है जो देश के दूरदराज के हिस्सों में हाई-स्पीड इंटरनेट कनेक्टिविटी प्रदान करेगा।'''
article2 = '''भारत सरकार ने ट्विटर और फेसबुक जैसे सोशल मीडिया प्लेटफॉर्म के लिए नए नियम जारी किए हैं, जिसमें उन्हें उपयोगकर्ता शिकायतों को संभालने और एक विशिष्ट समय सीमा के भीतर कुछ प्रकार की सामग्री को हटाने के लिए अधिकारी नियुक्त करने की आवश्यकता है।
नए नियमों को ट्विटर और फेसबुक दोनों से आलोचना का सामना करना पड़ा है, जो तर्क देते हैं कि वे बोलने की स्वतंत्रता को खतरे में डालते हैं और असंतोष की आवाज़ों को दबाने के लिए इस्तेमाल किया जा सकता है।
नए नियमों को लेकर विवाद के बावजूद, भारत सरकार ने कहा है कि सोशल मीडिया प्लेटफॉर्म पर फर्जी खबरों और अभद्र भाषा के प्रसार को रोकने के लिए ये आवश्यक हैं।'''

In [ ]:
def translate(article):
  article_list = article.split("।")
  article_list = [sentence.replace("\n",'') for sentence in article_list]
  translated_text = [translator(sentence)[0]['translation_text'] for sentence in article_list if len(sentence)>0]
  return '.'.join(translated_text)

In [ ]:
translated_article = translate(article2)

In [ ]:
translated_article

'The Government of India has issued new rules for a social media platform such as Twitter and Facebook in which they need to appoint officers to sustain user complaints and remove certain items within a specific period of time.The new rules have had to deal with criticism both from Twitter and Facebook who argue that they threaten the freedom of speech and may be used to suppress the voices of nationalism.In spite of a dispute with new rules the Government of India has said that these are essential to prevent the spread of information and infrastructure on social media platforms'